In [4]:
import pandas as pd
import os

carpeta = "Data encontrada"

# Lista para almacenar cada DataFrame
dataframes = []

# Iterar sobre todos los archivos en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".xlsx") or archivo.endswith(".xls"):
        ruta = os.path.join(carpeta, archivo)
        print(f"Leyendo: {archivo}")
        df = pd.read_excel(ruta)
        nombre_archivo = os.path.splitext(archivo)[0]  # Quita la extensión
        df["Pagina"] = nombre_archivo.replace("resultados_", "")
        df["Pagina"] = df["Pagina"].str.upper()
        dataframes.append(df)



#buscar repuestos en el lider


df_concatenado = pd.concat(dataframes, ignore_index=True)


df_concatenado = df_concatenado.drop_duplicates(subset=['Nombre Producto', 'Link'])


df_concatenado['Precio'] = (
    df_concatenado['Precio']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace('.', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
)

# 2. Convertir a número (manteniendo NaN si no se puede convertir)
df_concatenado['Precio'] = pd.to_numeric(df_concatenado['Precio'], errors='coerce')

# 3. Guardar todos los que son NaN o 0
df_precios_cero = df_concatenado[df_concatenado['Precio'].isna() | (df_concatenado['Precio'] == 0)].copy()

# 4. Eliminar del DataFrame original
df_concatenado = df_concatenado[~df_concatenado.index.isin(df_precios_cero.index)].copy()


mask_dup = df_concatenado.duplicated(subset=['Nombre Producto'], keep=False)
df_nombres_duplicados = df_concatenado[mask_dup].copy()
df_concatenado = df_concatenado[~mask_dup].copy()


df_nombres_repetidos = df_nombres_duplicados[~df_nombres_duplicados["Pagina"].isin(["MUNDOREPUESTOS","CHILEREPUESTOS","CIPER",])]
df_nombres_duplicados = df_nombres_duplicados[df_nombres_duplicados["Pagina"].isin(["MUNDOREPUESTOS","CHILEREPUESTOS","CIPER",])] 


df_nombres_repetidos.to_excel('Data consolidada/repetidos_sin_modificar.xlsx')
df_nombres_duplicados.to_excel('Data consolidada/df_nombres_duplicados.xlsx', index=False)
df_concatenado.to_excel('Data consolidada/consolidado.xlsx', index=False)
df_precios_cero.to_excel('Data consolidada/repuestos_sin_precio.xlsx', index=False)





Leyendo: resultados_autoplanet.xlsx
Leyendo: resultados_casaderepuestos.xlsx
Leyendo: resultados_chilerepuestos.xlsx
Leyendo: resultados_ciper.xlsx
Leyendo: resultados_emgi.xlsx
Leyendo: resultados_inalco.xlsx
Leyendo: resultados_mundorepuestos.xlsx
Leyendo: resultados_salfarepuestos.xlsx
Leyendo: resultados_takora.xlsx
Leyendo: resultados_ulti.xlsx
